In [2]:
import os
import warnings
import platform
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

%reload_ext nb_black
%config IPCompleter.greedy=True

<IPython.core.display.Javascript object>

In [3]:
# /home/ryd4/DeepRoute/protoc/bin/protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . --user

<IPython.core.display.Javascript object>

In [4]:
CUSTOM_MODEL_NAME = "my_ssd_mobnet"
PRETRAINED_MODEL_NAME = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
PRETRAINED_MODEL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
TF_RECORD_SCRIPT_NAME = "generate_tfrecord.py"
LABEL_MAP_NAME = "label_map.pbtxt"
current_os = platform.system()

<IPython.core.display.Javascript object>

In [5]:
labels = [{"name": "parent-mice", "id": 1}, {"name": "child-mice", "id": 2}]

<IPython.core.display.Javascript object>

In [6]:
parent_directory = os.path.dirname(os.getcwd())
paths = {
    "SRC_PATH": os.path.join(parent_directory, "src"),
    "DATA_PATH": os.path.join(parent_directory, "data"),
    "APIMODEL_PATH": os.path.join(parent_directory, "api_models"),
    "MODEL_PATH": os.path.join(parent_directory, "my_models"),
    "PRETRAINED_MODEL_PATH": os.path.join(parent_directory, "pre-trained-models"),
    "CHECKPOINT_PATH": os.path.join(parent_directory, "my_models", CUSTOM_MODEL_NAME),
    "OUTPUT_PATH": os.path.join(
        parent_directory, "my_models", CUSTOM_MODEL_NAME, "export"
    ),
    "TFJS_PATH": os.path.join(
        parent_directory, "my_models", CUSTOM_MODEL_NAME, "tfjsexport"
    ),
    "TFLITE_PATH": os.path.join(
        parent_directory, "my_models", CUSTOM_MODEL_NAME, "tfliteexport"
    ),
    "PROTOC_PATH": os.path.join(parent_directory, "protoc"),
}

<IPython.core.display.Javascript object>

In [7]:
files = {
    "PIPELINE_CONFIG": os.path.join(
        parent_directory, "my_models", CUSTOM_MODEL_NAME, "pipeline.config"
    ),
    "TF_RECORD_SCRIPT": os.path.join(paths["DATA_PATH"], TF_RECORD_SCRIPT_NAME),
    "LABELMAP": os.path.join(paths["DATA_PATH"], LABEL_MAP_NAME),
}

<IPython.core.display.Javascript object>

In [8]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == "posix":
            !mkdir -p {path}
        if os.name == "nt":
            !mkdir {path}

<IPython.core.display.Javascript object>

In [9]:
if not os.path.exists(
    os.path.join(paths["APIMODEL_PATH"], "research", "object_detection")
):
    !cd {paths['APIMODEL_PATH']} && module load git && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


<IPython.core.display.Javascript object>

In [10]:
download_model = True
if download_model and os.name == "posix":
    !wget {PRETRAINED_MODEL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if download_model and os.name == "nt":
    !wget.download(PRETRAINED_MODEL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2021-07-02 00:44:53--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.7.16, 2607:f8b0:4006:800::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.7.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

100%[======================================>] 20,515,344  63.7MB/s   in 0.3s   

2021-07-02 00:44:53 (63.7 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/

<IPython.core.display.Javascript object>

In [11]:
# # Install Tensorflow Object Detection
# if current_os == "Linux":
#     !apt-get install protobuf-compiler
#     !cd {f'{paths['APIMODEL_PATH']}/models/research'} && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install {.}
# if current_os == "Windows":
#     url = "https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
#     wget.download(url)
#     !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
#     !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
#     os.environ["PATH"] += os.pathsep + os.path.abspath(
#         os.path.join(paths["PROTOC_PATH"], "bin")
#     )
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
#     !cd Tensorflow/models/research/slim && pip install -e {.}
# if current_os == "Darwin":
#     api_model = f'{paths["APIMODEL_PATH"]}/models/research'
#     #!brew install protobuf-compiler
#     # !cd {api_model} && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

<IPython.core.display.Javascript object>

In [12]:
path_to_train = f'{paths["DATA_PATH"]}/train'
path_to_test = f'{paths["DATA_PATH"]}/test'
path_to_labels = f'{paths["DATA_PATH"]}/{LABEL_MAP_NAME}'

<IPython.core.display.Javascript object>

In [13]:
# !python3 generate_tfrecord.py -x {path_to_train} -l {path_to_labels} -o {os.path.join(paths["DATA_PATH"], 'train.record')}
# !python3 generate_tfrecord.py -x {path_to_test} -l {path_to_labels} -o {os.path.join(paths["DATA_PATH"], 'test.record')}

<IPython.core.display.Javascript object>

In [14]:
if os.name == "posix":
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == "nt":
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

<IPython.core.display.Javascript object>

In [15]:
config = config_util.get_configs_from_pipeline_file(files["PIPELINE_CONFIG"])
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

<IPython.core.display.Javascript object>

In [16]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files["PIPELINE_CONFIG"], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

<IPython.core.display.Javascript object>

In [17]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(
    paths["PRETRAINED_MODEL_PATH"], PRETRAINED_MODEL_NAME, "checkpoint", "ckpt-0"
)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = files["LABELMAP"]
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths["DATA_PATH"], "train.record")
]
pipeline_config.eval_input_reader[0].label_map_path = files["LABELMAP"]
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths["DATA_PATH"], "test.record")
]

<IPython.core.display.Javascript object>

In [18]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files["PIPELINE_CONFIG"], "wb") as f:
    f.write(config_text)

<IPython.core.display.Javascript object>

In [19]:
config = config_util.get_configs_from_pipeline_file(files["PIPELINE_CONFIG"])
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

<IPython.core.display.Javascript object>

In [20]:
TRAINING_SCRIPT = os.path.join(
    paths["APIMODEL_PATH"],
    "models",
    "research",
    "object_detection",
    "model_main_tf2.py",
)
command = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(
    TRAINING_SCRIPT, paths["CHECKPOINT_PATH"], files["PIPELINE_CONFIG"]
)
print(command)

python3 /cache/home/ryd4/DeepRoute/api_models/models/research/object_detection/model_main_tf2.py --model_dir=/cache/home/ryd4/DeepRoute/my_models/my_ssd_mobnet --pipeline_config_path=/cache/home/ryd4/DeepRoute/my_models/my_ssd_mobnet/pipeline.config --num_train_steps=2000


<IPython.core.display.Javascript object>

In [22]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(
    TRAINING_SCRIPT,
    paths["CHECKPOINT_PATH"],
    files["PIPELINE_CONFIG"],
    paths["CHECKPOINT_PATH"],
)
print(command)

python /cache/home/ryd4/DeepRoute/api_models/models/research/object_detection/model_main_tf2.py --model_dir=/cache/home/ryd4/DeepRoute/my_models/my_ssd_mobnet --pipeline_config_path=/cache/home/ryd4/DeepRoute/my_models/my_ssd_mobnet/pipeline.config --checkpoint_dir=/cache/home/ryd4/DeepRoute/my_models/my_ssd_mobnet


<IPython.core.display.Javascript object>